In [3]:
user_name = 'celiaberon'

In [113]:
import sys
sys.path.append('/Users/{}/GitHub/mouse_bandit/jupyter_notebooks/helper_functions'.format(user_name))
import hf_behavior_imaging as hf
import numpy as np
import itertools

import pandas as pd
import os
sys.path.append('/Users/{}/GitHub/mouse_bandit/data_preprocessing_code'.format(user_name))
import bandit_preprocessing as bp
import scipy.io as scio

In [ ]:
cond1_name = 'Reward'; cond1_id=[0,1]
cond1_ops = '='

cond2_name = 'Decision'; cond2_id=[0,1]

cond3_name = 'Reward'; cond3_id=[0,1]


if 'cond1_ops' in vars():
    cond1_ops_b='=' # make sure the alternative operation fits with its partner 
    if cond1_ops != '=':
        if cond1_ops == '>':
            cond1_ops_b = '<='
            print(cond1_ops_b)
        elif cond1_ops == '>=':
            cond1_ops_b = '<'
            print(cond1_ops_b)

In [114]:
ca_data_path = '/Users/{}/Documents/mouse_bandit/k13_03102017_bandit_8020_neuron_results.mat'.format(user_name)
ca_data = scio.loadmat(ca_data_path,squeeze_me = True, struct_as_record = False)
neuron = ca_data['neuron_results'] # set neuron variable to just contain neuron_results structure

record_path = '/Users/{}/GitHub/mouse_bandit/session_record.csv'.format(user_name) # session record of behavioral data

record = pd.read_csv(record_path,index_col=0) # session record is panda df

'''Specify mouse and session of interest'''

session_name  = '03102017_K13'
mouse_id = 'k13'

record[record['Session ID'] == session_name] # take only rows from record that match session name


'''Load in trial data from specific session'''

columns = ['Elapsed Time (s)','Since last trial (s)','Trial Duration (s)','Port Poked',
           'Right Reward Prob','Left Reward Prob','Reward Given',
          'center_frame','decision_frame']

root_dir = '/Users/{}/GitHub/mouse_bandit/data/trial_data'.format(user_name)
full_name = session_name + '_trials.csv'

path_name = os.path.join(root_dir,full_name)

trial_df = pd.read_csv(path_name,names=columns)

feature_matrix = bp.create_feature_matrix(trial_df,10,mouse_id,session_name,feature_names='Default',imaging=True)

In [134]:
def align_frames(df, ca_data, event, cond1_name, cond1=False, cond2_name=False,cond2=False, cond3_name=False,
                   cond3=False, cond1_ops= '=', cond2_ops = '=', cond3_ops = '=', extension=30):
    
    conditions = [cond1_name, cond2_name, cond3_name]
    n_variables = 3 - conditions.count(0) # value between 1 and 3 to run the corresponding number of conditions through the rest of the notebook
    combo_id = list(itertools.product([0, 1], repeat=n_variables)) # list of all combinations of binary conditions
    n_combos = len(combo_id) # total number of combinations of conditions
    
    poke_frames = {} # initialize an empty dictionary to store center and decision poke frames for each combination of conditions
    for i in range(0,n_combos):
        if n_variables==3: # based on number of variables specified
            iCombo_key= "f%s%s%s" % (combo_id[i][0], combo_id[i][1], combo_id[i][2]) # set key name based on comboID
            iCombo_value = hf.extract_frames(df, cond1_name, combo_id[i][0], cond2_name, 
                                             combo_id[i][1], cond3_name, combo_id[i][2], cond1_ops=cond1_ops)
            temp_dict = {iCombo_key:iCombo_value} # create new key-value pair with comboID and corresponding frames
            poke_frames.update(temp_dict) # add new values to dictionary of center and decision poke frames
        if n_variables==2:
            iCombo_key= "f%s%s" % (combo_id[i][0], combo_id[i][1])
            iCombo_value = hf.extract_frames(df, cond1_name, combo_id[i][0], cond2_name, 
                                             combo_id[i][1], cond1_ops=cond1_ops)
            temp_dict = {iCombo_key:iCombo_value}
            poke_frames.update(temp_dict)
        if n_variables==1:
            iCombo_key= "f%s" %(combo_id[i][0])
            iCombo_value = hf.extract_frames(df, cond1_name, combo_id[i][0], cond1_ops=cond1_ops)
            temp_dict = {iCombo_key:iCombo_value}
            poke_frames.update(temp_dict)

    poke_frames_keys = list(poke_frames.keys())
    
    for i in poke_frames: # create full window based on number of frames in 'extension' variable
        poke_frames[i][:,0] = poke_frames[i][:,0] - extension
        poke_frames[i][:,1] = poke_frames[i][:,1] + extension
    
    nTrials = [poke_frames[poke_frames_keys[i]].shape[0] for i in range(n_combos)]
    max_window = np.zeros(n_combos) 
    window_length= np.zeros((np.max(nTrials), n_combos))

    for i in range(n_combos):
        for iTrial in range(nTrials[i]):
            window_length[iTrial, i] = int(((poke_frames[poke_frames_keys[i]][iTrial][1]-
                                     poke_frames[poke_frames_keys[i]][iTrial][0])))
        max_window[i] = np.max(window_length)

    max_window = int(max_window.max())
    
    nNeurons = ca_data.shape[0]
    nFrames = ca_data.shape[1]
    
    aligned_traces = np.empty((np.max(nTrials), max_window, nNeurons, n_combos))

    if event == 'centerPoke':
        for i in range(n_combos):
            
            # create array containing segment of raw trace for each neuron for each trial aligned to center poke
            for iNeuron in range(nNeurons): # for each neuron
                for iTrial in range(0,nTrials[i]): # and for each trial
                    try:
                        aligned_traces[iTrial,:, iNeuron, i] = ca_data[iNeuron,
                            int(poke_frames[poke_frames_keys[i]][iTrial][0]):
                            (int(poke_frames[poke_frames_keys[i]][iTrial][0])+max_window)]
                    except ValueError: # for trials at end of session where cannot fill full max_window length
                        len_to_end = int(nFrames - poke_frames[poke_frames_keys[i]][iTrial][0])
                        aligned_traces[iTrial,0:len_to_end, iNeuron, i] = ca_data[iNeuron,
                            int(poke_frames[poke_frames_keys[i]][iTrial][0]):
                            (int(poke_frames[poke_frames_keys[i]][iTrial][0])+len_to_end)]

    elif event == 'decisionPoke':
        for i in range(n_combos):

            # create array containing segment of raw trace for each neuron for each trial aligned to decision poke
            for iNeuron in range(nNeurons): # for each neuron
                for iTrial in range(0,nTrials[i]): # and for each trial
                    try:
                        aligned_traces[iTrial,:, iNeuron, i] = ca_data[iNeuron,
                            int(poke_frames[poke_frames_keys[i]][iTrial][0]):
                            (int(poke_frames[poke_frames_keys[i]][iTrial][0])+max_window)]
                    except ValueError: # for trials at end of session where cannot fill full max_window length
                        len_to_end = int(nFrames - poke_frames[poke_frames_keys[i]][iTrial][0])
                        #aligned_start[iTrial,0:len_to_end, iNeuron, i] = normalized[iNeuron,
                        #    int(poke_frames[poke_frames_keys[i]][iTrial][0]):
                        #    (int(poke_frames[poke_frames_keys[i]][iTrial][0])+len_to_end)]
                        aligned_traces[iTrial,0:len_to_end, iNeuron, i] = 'NaN'

    return poke_frames_keys, aligned_traces, nTrials

In [133]:
align_frames(feature_matrix, neuron.C_raw, 'decisionPoke', cond1_name='Reward', cond2_name='Decision')

(['f00', 'f01', 'f10', 'f11'],
 array([[[[ -1.27731759e+00,  -5.48994920e+00,  -8.81815553e-01,
            -1.48423490e+00],
          [  7.97704331e-01,  -7.45774444e-01,   8.67545298e-01,
            -1.52323088e+00],
          [ -1.59160600e+00,  -8.55439524e-01,   1.38410388e+00,
             4.54711810e-02],
          ..., 
          [  1.28322144e+00,   1.30759373e+00,  -1.74870589e+00,
            -1.38355853e+00],
          [  2.30195874e+00,  -2.42914991e+00,  -4.25100955e-01,
            -9.83990092e-01],
          [  2.17420270e-01,  -1.54487393e+00,   1.27522231e+00,
            -2.18603883e-01]],
 
         [[ -1.28070541e-01,  -2.72314405e+00,   1.14696236e-01,
            -1.51026049e+00],
          [  3.41499781e-01,   3.62194710e-01,  -1.05769850e+00,
            -2.46224944e+00],
          [ -1.07148983e+00,   5.45278812e-02,  -1.45134775e+00,
            -2.76182231e+00],
          ..., 
          [ -1.07261815e+00,   1.42560074e+00,  -3.37646234e+00,
            -4